# Description

This notebook only illustrates the workings of the auxiliary TS_Mixer model including static and dynamic features.

# Imports

In [1]:
from TSMixer.model_auxiliary import *

# Sample Data

In [2]:
torch.manual_seed(123)
fcst_h = 20
out_dim = 2
x = torch.randn(100, 3)[None, :]
z = torch.randn(fcst_h, 2)[None, :]
s = torch.ones_like(z)
s[:, :, 1] = 2

In [3]:
n_ts = x.shape[2]
n_static_feat = s.shape[2]
n_dynamic_feat = z.shape[2]
ts_length = x.shape[1]
embed_dim = 64

n_feat_sx = embed_dim + n_ts
n_feat_sz = embed_dim + n_dynamic_feat

In [4]:
mlp_x_map = nn.Linear(ts_length, fcst_h)
x_map = mlp_x_map(x.transpose(1, 2)).transpose(1, 2)

mlp_sx = MLP_Feat(n_static_feat, embed_dim, dropout=0.1)
sx_out = mlp_sx(s)

mlp_sz = MLP_Feat(n_dynamic_feat, embed_dim, dropout=0.1)
sz_out = mlp_sz(s)

mlp_x = MLP_Feat(n_feat_sx, embed_dim, dropout=0.1)
x_prime = mlp_x(torch.cat([x_map, sx_out], dim=2))

mlp_z = MLP_Feat(n_feat_sz, embed_dim, dropout=0.1)
z_prime = mlp_z(torch.cat([z, sz_out], dim=2))

y_prime = torch.cat([x_prime, z_prime], dim=2)

n_feat = embed_dim * 3

mixer_block = Mixer_Block(n_feat, n_static_feat, fcst_h, embed_dim, dropout=0.1)
y_prime_mixer = mixer_block(y_prime, s)

mlp_out = nn.Linear(embed_dim * 2, out_dim)
x_out = mlp_out(y_prime_mixer)

layer_norm = nn.LayerNorm(out_dim)
x_out = layer_norm(x_out)

x_map.shape, sx_out.shape, sz_out.shape, x_prime.shape, z_prime.shape, y_prime.shape, y_prime_mixer.shape, x_out.shape

(torch.Size([1, 20, 3]),
 torch.Size([1, 20, 64]),
 torch.Size([1, 20, 64]),
 torch.Size([1, 20, 64]),
 torch.Size([1, 20, 64]),
 torch.Size([1, 20, 128]),
 torch.Size([1, 20, 128]),
 torch.Size([1, 20, 2]))

# TS-Mixer

In [5]:
num_blocks = 2
out_dim = 2
ts_mixer_aux = TS_Mixer_auxiliary(
    n_ts,
    n_static_feat,
    n_dynamic_feat,
    ts_length,
    embed_dim,
    num_blocks,
    fcst_h,
    out_dim,
    dropout=0.1,
)
out = ts_mixer_aux(x, z, s)
print(ts_mixer_aux)

TypeError: forward() takes 2 positional arguments but 3 were given